In [1]:
import pandas as pd
import numpy as np
import pickle

from tqdm import tqdm

from models import Model2
from tools import Train, validation, GridSearch

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')

from sklearn.metrics import r2_score, mean_absolute_error

In [3]:
with open('data/train_dataset.pickle', 'rb') as f:
    train_dataset = pickle.load(f)

with open('data/val_dataset.pickle', 'rb') as f:
    val_dataset = pickle.load(f)

with open('data/test_dataset.pickle', 'rb') as f:
    test_dataset = pickle.load(f)

In [4]:
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 64, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 사용법

+ `basic_params`: 딱히 건드릴 필요 없는 애들 (`n` 같은 것들)
+ `param_grid`: 그리드 서치의 대상이 되는 애들 (`m`: LSTM hidden size, `skip_hidden_size`: Skip GRU hidden size 등)
+ 기본적으로 Train과 사용법이 비슷함
    > `Train.best_model`: epoch들 중에서 loss가 가장 낮았던 순간을 기억  
    > `GridSearch.best_model`: parameter 조합들에 의한 여러 model들 중에서 loss가 가장 낮았던 것을 기억  
    > 아래 나오는 플롯들은 모두 최적의 parameter의 model에 대한 플롯

In [5]:
criterion = nn.MSELoss()
eval_metrics = [
    r2_score,
    mean_absolute_error
]

# Model 2

In [6]:
basic_params = {
    'T': 60,
    'n': 40
}

param_grid = {
    'm': [32, 40],
    'skip_hidden_size': [32, 40],
    'T_modified': [20, 30],
    'skip': [8, 10]
}

In [7]:
grid_model = GridSearch(criterion, eval_metrics, device, temp_save_path='checkpoints/model_by_param/temp_model.pt', masked=False)
best_model = grid_model.train_by_grid(
        Model2, basic_params, param_grid,
        torch.optim.Adam, train_loader, val_loader,
        patience=3, epochs=20,
        save_filename='checkpoints/model2_best.pt'
    )

Total  16 cases are going to be searched.
Setting   0 of Parameters Grid is now on progres.


Epoch 13: 100%|██████████| 2607/2607 [02:36<00:00, 16.67it/s, Train Loss=0.6084, Valid Loss=0.2702, Valid R2=0.3904, Valid MAE=0.4176]


Early Stopped
Setting   1 of Parameters Grid is now on progres.


Epoch  6: 100%|██████████| 2607/2607 [02:36<00:00, 16.61it/s, Train Loss=0.6421, Valid Loss=0.1921, Valid R2=0.5666, Valid MAE=0.3407]


Early Stopped
Setting   2 of Parameters Grid is now on progres.


Epoch 13: 100%|██████████| 2607/2607 [02:54<00:00, 14.91it/s, Train Loss=0.5849, Valid Loss=0.4194, Valid R2=0.0537, Valid MAE=0.4774]


Early Stopped


RuntimeError: Error(s) in loading state_dict for Model2:
	Missing key(s) in state_dict: "layer_cnn.0.weight", "layer_cnn.0.bias", "layer_cnn_beta1.0.weight", "layer_cnn_beta1.0.bias", "linear_beta1.0.weight", "linear_beta1.0.bias". 
	Unexpected key(s) in state_dict: "layer_linear_beta1.0.weight", "layer_linear_beta1.0.bias". 

In [ ]:
best_model.eval()
with torch.no_grad():
    loss, r2, mae = validation(best_model, val_loader, criterion, eval_metrics, device)
    print(f'Loss={loss:.4f}, R2={r2:.4f}, MAE={mae:.4f}')

In [ ]:
grid_model.test(test_loader)

In [ ]:
grid_model.plot_losses(plot_title='Model2.png', save_filename='Model2_losses.png')